# Intro
- This is the first jupyter notebook for the EIA interhemispheric assymmetry (IHA) seen during different solar cycles
- TEC comparison during the March equinox is carried out for different years
    - Peaks of the north and south EIA are traced and compared

# Reading New TEC Data and Producing Output Files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import cartopy.crs as ccrs
from tqdm import tqdm
from apexpy import Apex
import os
import datetime as dt
import importlib as il

from p_tqdm import p_map
from multiprocessing import Pool
import itertools

/work/10028/prasoonv/ls6/miniconda3/envs/jlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Choose the system: i -> 0:tacc, 1:mac, 2:ganymede
i = 0

if i == 0:
    work = '/work/10028/prasoonv/ls6/repo/sat-interp-tid-analysis/'
    scratch = '/scratch/10028/prasoonv/' 
elif i == 1:
    work = '/Users/prasoonv/repo/sat-interp-tid-analysis/Qingyu_Cesar_EIA_IHA/'
    scratch = '/Users/prasoonv/repo/sat-interp-tid-analysis/scratch/'
elif i == 2:
    work = '/home/pxv220016/prasoon/data/sat_interp_repo/sat-interp-tid-analysis/'
    scratch = '/home/pxv220016/scratch/'

import sys
sys.path.append('')
sys.path.append(f'{work}prasoon_utility_programs')
sys.path.append(f'{work}Qingyu_Cesar_EIA_IHA')
import functions

In [2]:
functions = il.reload(functions)
month = 'june' #'march'

if month == 'sept' or month == 'march':
    phase = 'equinox'
else:
    phase = 'solstice'

if month == 'sept':
    years_tot = list(range(2020, 2024))
else:
    years_tot = list(range(2020, 2025))
    
for year in tqdm(years_tot):
    
    print(year)

    
    # Reading Madrigal Cedar data for +- 21 days around March equinox of 2010-2024
    # Files in below scratch folder are obtained by using multiple file download 
    # command in ASCII format and then doing `gunzip file.gz`  
    path = f'{scratch}Qingyu_Cesar_EIA/' + month + '_data/' + str(year) + '_' + month + '_' + phase + '/'
    files = os.listdir(path)
    files = [path + i for i in files]

    for f in files:
        if f[-4:] != '.txt':
            files.remove(f)
            
    print(len(files))
    
    tec_g = []
    columns = ['GDLAT', 'GLON', 'TEC', 'DT']
    grnd_tec = pd.DataFrame(columns=columns)

    def process_file(f):
        # Read function passed continuously during multiple processing to quicken the process
        df = pd.read_csv(f, sep=r'\s+')
        d = [dt.datetime(y, m, d, h, mi, s) for y, m, d, h, mi, s in zip(df.YEAR, df.MONTH, df.DAY, df.HOUR, df.MIN, df.SEC)]
        df['DT'] = d
        # Dropping unnecassary columns from the Dataframe
        df = df.drop(['RECNO', 'KINDAT', 'KINST', 'UT1_UNIX', 'UT2_UNIX', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MIN', 'SEC', 'DTEC'], axis=1)
        if 'GDALT' in df.columns:
            df = df.drop(['GDALT'], axis=1)
        df = df[(df.GDLAT > -60) & (df.GDLAT < 60) & (df.GLON > -85) & (df.GLON < -45)].reset_index(drop=True)
        return df    
    # Speeding the process by using parallel processing
    tec_g = p_map(process_file, files)  # Parallel processing with progress bar
    print('1')
    grnd_tec = pd.concat(tec_g, axis=0).reset_index(drop=True)
    grnd_tec = grnd_tec.sort_values(by=['DT', 'GDLAT'], ascending=[True, True])


    # Reading the Kp index values for all the days and filtering undesired points where Kp > 3
    file = f'{work}Qingyu_Cesar_EIA_IHA/kp3_index_values/kp_{str(year)}_{month}.txt'
    kp = pd.read_csv(file,sep=r'\s+')
    date_kp = [functions.day_to_date(i, year) for i in kp.DOY]
    m, d = zip(*date_kp)
    kp['date'] = [dt.datetime(year, j, i, k, 0, 0) for i,j,k in zip(d,m,kp.Hour)]
    kp['kp'] = [i/10 for i in kp.Kp]
    kp = kp.drop(['Year', 'DOY', 'Hour','Kp'], axis = 1)
    grnd_tec0 = functions.kp_index_filtering(grnd_tec, kp)
    print('2')

    # Calculation of magnetic coordinates by using Apex library and Parallel prcoessing 
    t_start = dt.datetime.now() # just a timer
    with Pool(24) as pool:
        p = pool.starmap(functions.magnetic_coords_parallel, zip(grnd_tec0.DT, grnd_tec0.GDLAT, grnd_tec0.GLON, grnd_tec0.TEC))
    pool.close()
    pool.join()
    # Separating the data from output list 
    sat_date, sat_glat, sat_glon, sat_tec, sat_mlat, sat_mlon, sat_mlt = zip(*p)
    t_total = dt.datetime.now() - t_start
    print(t_total)
    

    # Reordering the outputs and applying further conditions on magnetic coordinates
    grnd_temp = pd.DataFrame({'DT': sat_date, 'GDLAT': sat_glat, 'GLON': sat_glon, 'TEC': sat_tec, 'MLAT': sat_mlat, 'MLON': sat_mlon, 'MLT': sat_mlt})
    grnd_tec1 = grnd_temp.sort_values(by=['DT', 'GDLAT'], ascending=[True, True]).reset_index()
    grnd_tec1 = grnd_tec1[(grnd_tec1.MLON <= 5) & (grnd_tec1.MLON >= -5)].reset_index(drop=True)
    grnd_tec1 = grnd_tec1[(grnd_tec1.MLAT <= 40) & (grnd_tec1.MLAT >= -40)].reset_index(drop=True)
    grnd_tec2 = grnd_tec1.drop(['GDLAT', 'GLON','MLON'], axis = 1)

    
    # Writing the output into csv files for easy post processing
    grnd_tec2.to_csv(f'{work}Qingyu_Cesar_EIA_IHA/outputs/{month}/{str(year)}_{month}_{phase}.csv', index=False)


  0%|                                                     | 0/4 [00:00<?, ?it/s]

2021
43


  0%|          | 0/43 [00:00<?, ?it/s]

1
2
0:17:02.089608


 25%|██████████▎                              | 1/4 [21:30<1:04:32, 1290.90s/it]

...

2024
43


IOStream.flush timed out


  0%|          | 0/43 [00:00<?, ?it/s]

1
2
0:13:38.309408


100%|█████████████████████████████████████████| 4/4 [1:50:24<00:00, 1656.20s/it]


# F10.7 Comparison for March 2023 and 2024

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import cartopy.crs as ccrs
from tqdm import tqdm
from apexpy import Apex
import datetime as dt


In [ ]:
# Choose the system: i -> 0:tacc, 1:mac, 2:ganymede
i = 0

if i == 0:
    work = '/work/10028/prasoonv/ls6/repo/sat-interp-tid-analysis/'
    scratch = '/scratch/10028/prasoonv/' 
elif i == 1:
    work = '/Users/prasoonv/repo/sat-interp-tid-analysis/Qingyu_Cesar_EIA_IHA/'
    scratch = '/Users/prasoonv/repo/sat-interp-tid-analysis/scratch/'
elif i == 2:
    work = '/home/pxv220016/prasoon/data/sat_interp_repo/sat-interp-tid-analysis/'
    scratch = '/home/pxv220017/scratch/'

import sys
sys.path.append('')
sys.path.append(f'{work}prasoon_utility_programs')
sys.path.append(f'{work}Qingyu_Cesar_EIA_IHA')
import functions

In [ ]:
functions = il.reload(functions)
f107_years = [2023, 2024]
month = 'march'

path = f'{work}Qingyu_Cesar_EIA_IHA/outputs/{month}/f107_{month}_'

columns = ['Year', 'DOY', 'Hour', 'f10_7']
f107 = pd.DataFrame(columns = columns)

for y in f107_years:
    file = f'{path}{str(y)}.txt'
    df_f107 = pd.read_csv(file, sep=r'\s+')
    print(y, 'year -> Mean =', sum(df_f107.f10_7)/len(df_f107.f10_7), 'and Median =', np.median(df_f107.f10_7))
print('\nBoth mean and median of F10.7 index during ' + month + ' equinox for 2023 is higher than 2024.')

2023 year -> Mean = 150.87441860465202 and Median = 148.1
2024 year -> Mean = 146.01395348837212 and Median = 137.6

Both mean and median of F10.7 index during march equinox for 2023 is higher than 2024.


# Reading Data Directly From Saved Output Files

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import cartopy.crs as ccrs
from tqdm import tqdm
from apexpy import Apex
import datetime as dt
from scipy.signal import savgol_filter


In [5]:
# Choose the system: i -> 0:tacc, 1:mac, 2:ganymede
i = 0

if i == 0:
    work = '/work/10028/prasoonv/ls6/repo/sat-interp-tid-analysis/'
    scratch = '/scratch/10028/prasoonv/' 
elif i == 1:
    work = '/Users/prasoonv/repo/sat-interp-tid-analysis/Qingyu_Cesar_EIA_IHA/'
    scratch = '/Users/prasoonv/repo/sat-interp-tid-analysis/scratch/'
elif i == 2:
    work = '/home/pxv220016/prasoon/data/sat_interp_repo/sat-interp-tid-analysis/'
    scratch = '/home/pxv220016/scratch/'

import sys
sys.path.append('')
sys.path.append(f'{work}prasoon_utility_programs')
sys.path.append(f'{work}Qingyu_Cesar_EIA_IHA')
import functions

In [6]:
month = 'march'

if month == 'sept' or month == 'march':
    phase = 'equinox'
else:
    phase = 'solstice'

if month == 'sept':
    years = list(range(2000, 2024))
else:
    years = list(range(2000, 2025))


for year in tqdm(years):
    # Reading the data from output csv
    grnd_tec2 = pd.read_csv(f'{scratch}Qingyu_Cesar_EIA/outputs/{month}/{str(year)}_{month}_{phase}.csv')


    # Define the bin edges for MLAT and MLT
    bins_mlat = pd.cut(grnd_tec2['MLAT'], bins=pd.interval_range(start=-40, end=40, freq=1))
    bins_mlt = pd.cut(grnd_tec2['MLT'], bins=pd.interval_range(start=0, end=24, freq=0.25))
    
    # Create a new DataFrame with the bins
    grnd_tec2['MLAT_b'] = bins_mlat
    grnd_tec2['MLT_b'] = bins_mlt
    
    # Group by the bins (MLAT_b (primary) and MLT_b (secondary)) and calculate the average of TEC
    result = grnd_tec2.groupby(['MLT_b', 'MLAT_b'])['TEC'].mean().reset_index()
    # Converting the midpoint values of bins to float and assigning average TEC at those points
    result['MLAT_b'] = result['MLAT_b'].apply(lambda x: x.mid)
    result['MLT_b'] = result['MLT_b'].apply(lambda x: x.mid)
    result['MLAT_b'] = result['MLAT_b'].astype(float)
    result['MLT_b'] = result['MLT_b'].astype(float)


    result = result[result.MLT_b >= 5].reset_index(drop=True)
    
    
    filtered = []
    for t in result['MLT_b'].unique():
        result_f = result[result['MLT_b'] == t].reset_index(drop=True)
        fit = savgol_filter(np.array(result_f.TEC), 10, 2)
        filtered.extend(fit)
    result['TEC'] = filtered
    result = result.groupby(['MLAT_b', 'MLT_b']).sum().reset_index()
    
    
    # Identifying NH and SH peaks
    result_t = result[result.MLT_b >= 13].reset_index(drop=True)
    result_n = result_t[result_t.MLAT_b > 0].reset_index(drop=True)
    result_s = result_t[result_t.MLAT_b < 0].reset_index(drop=True)
    result_n = result_n.loc[result_n.groupby('MLT_b')['TEC'].idxmax()].reset_index(drop=True)
    result_s = result_s.loc[result_s.groupby('MLT_b')['TEC'].idxmax()].reset_index(drop=True)
    # Dropping the cases where the SH peak is not prominent and maxima appears to come at equator
    result_n.loc[result_n['MLAT_b'] < 5, 'TEC'] = np.nan
    result_s.loc[result_s['MLAT_b'] > -5, 'TEC'] = np.nan
    #print(result_s)
    
    i = 100*2*(result_n.TEC - result_s.TEC)/(result_n.TEC + result_s.TEC)
    result_ind = pd.DataFrame({'mlat_n': result_n.MLAT_b, 'tec_n': result_n.TEC, 'mlat_s': result_s.MLAT_b, 'tec_s': result_s.TEC, 'mlt': result_n.MLT_b, 'asy': i})
    result_ind.to_csv(f'{work}Qingyu_Cesar_EIA_IHA/outputs/{month}/asy_{str(year)}_{month}.csv', index=False)

    X, Y = np.meshgrid(result.MLT_b.unique(), result.MLAT_b.unique())
    Z = result.TEC.values.reshape(X.shape)
    if np.max(Z) <= 40:
        contour_levels = list(range(0, 41, 5))
    elif np.max(Z) <= 100:
        contour_levels = list(range(0, 101, 5))
    elif np.max(Z) <= 120:
        contour_levels = list(range(0, 121, 5))
    elif np.max(Z) <= 150:
        contour_levels = list(range(0, 151, 5))

    # Creating new dataframe to remove rows where TEC= NaN
    result_n = pd.DataFrame({'mlt': result_ind.mlt, 'mlat_n': result_ind.mlat_n, 'tec_n': result_ind.tec_n})
    result_n = result_n.dropna(subset='tec_n')
    result_s = pd.DataFrame({'mlt': result_ind.mlt, 'mlat_s': result_ind.mlat_s, 'tec_s': result_ind.tec_s})
    result_s = result_s.dropna(subset='tec_s')

    
    fig = plt.figure(figsize=(12,4))
    specs = fig.add_gridspec(1, 2, width_ratios=[1,0.1])
    ax = []
    ax.append(fig.add_subplot(specs[0, 0]))
    c = ax[0].contourf(X, Y, Z, levels=contour_levels, cmap = 'jet')
    ax[0].scatter(result_n.mlt, result_n.mlat_n, c='b', s=30)
    ax[0].scatter(result_s.mlt, result_s.mlat_s, c='b', s=30)
    ax[0].set_title(f'Tracing EIA Peaks - {str(year)} {month} {phase} (Bins -> MLAT - 1 deg, MLT = 0.25 hour)')
    ax[0].set_ylabel('MLAT')
    ax[0].set_xlabel('MLT')
    ax[0].grid(True)

    # Set ylim conditionally
    if np.max(Z) <= 40:
        ax[0].set_ylim(top=40)
    
    cbar_ax = fig.add_subplot(specs[0,1])
    cbar = fig.colorbar(c, cax=cbar_ax, label='VTEC (TECUnits)', extend='both')
    fig.savefig(f'{work}Qingyu_Cesar_EIA_IHA/outputs/eia_peaks/{month}/eia_peaks_{month}_{str(year)}.jpg')
    fig.show()




 80%|████████  | 20/25 [02:22<01:03, 12.77s/it]/tmp/ipykernel_1638304/4146714374.py:81: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(12,4))
100%|██████████| 25/25 [03:34<00:00,  8.60s/it]


# Comparing Asymmetry Index

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import cartopy.crs as ccrs
from tqdm import tqdm
from apexpy import Apex
import datetime as dt


In [8]:
# Choose the system: i -> 0:tacc, 1:mac, 2:ganymede
i = 0

if i == 0:
    work = '/work/10028/prasoonv/ls6/repo/sat-interp-tid-analysis/'
    scratch = '/scratch/10028/prasoonv/' 
elif i == 1:
    work = '/Users/prasoonv/repo/sat-interp-tid-analysis/Qingyu_Cesar_EIA_IHA/'
    scratch = '/Users/prasoonv/repo/sat-interp-tid-analysis/scratch/'
elif i == 2:
    work = '/home/pxv220016/prasoon/data/sat_interp_repo/sat-interp-tid-analysis/'
    scratch = '/home/pxv220017/scratch/'

import sys
sys.path.append('')
sys.path.append(f'{work}prasoon_utility_programs')
sys.path.append(f'{work}Qingyu_Cesar_EIA_IHA')
import functions

In [9]:
month = 'june' # 'march'

if month == 'sept' or month == 'march':
    phase = 'equinox'
else:
    phase = 'solstice'

if month == 'sept':
    years = list(range(2007, 2010)) + list(range(2018, 2021)) + list(range(2000, 2003)) + list(range(2012, 2015)) + list(range(2022, 2024))
else:
    years = list(range(2007, 2010)) + list(range(2018, 2021)) + list(range(2000, 2003)) + list(range(2012, 2015)) + list(range(2022, 2025))


asy_ind = []

for y in years:
    f = pd.read_csv(f'{work}Qingyu_Cesar_EIA_IHA/outputs/{month}/asy_{str(y)}_{month}.csv')
    asy_ind.append(f)

In [10]:
if month == 'sept':
    solar_cyc = [[list(range(2007, 2010))], [list(range(2018, 2021))], [list(range(2000, 2003))], [list(range(2012, 2015))], [list(range(2022, 2024))]]
else:
    solar_cyc = [[list(range(2007, 2010))], [list(range(2018, 2021))], [list(range(2000, 2003))], [list(range(2012, 2015))], [list(range(2022, 2025))]]


sc = ['Baseline Min-SC24', 'Baseline Min-SC25', 'Baseline Max-SC23', 'Baseline Max-SC24', 'Baseline Max-SC25']

sc_base = []

for s, n in tqdm(zip(solar_cyc, sc)):

    fig = plt.figure(figsize=(12,8))

    for periods in s:
        columns = ['mlt', 'asy']
        baseline = pd.DataFrame(columns=columns)

        for p in periods:
            a = pd.read_csv(f'{work}/Qingyu_Cesar_EIA_IHA/outputs/{month}/asy_{str(p)}_{month}.csv')
            a = a.drop(['mlat_n', 'mlat_s', 'tec_n', 'tec_s'], axis=1)
            #a = a.dropna(subset = 'asy')
            baseline = pd.concat([baseline, a], axis=0).reset_index(drop=True)
            plt.plot(a.mlt, a.asy, linestyle = '--', label = f'{str(p)} ASY Index')
        
        baseline = baseline.sort_values(by=['mlt'], ascending=[True])
        bins_mlt = pd.cut(baseline['mlt'], bins=pd.interval_range(start=0, end=24, freq=0.75))
        baseline['mlt'] = bins_mlt
        baseline = baseline.groupby(['mlt'])['asy'].mean().reset_index()
        baseline['mlt'] = baseline['mlt'].apply(lambda x: x.mid)
        baseline['mlt'] = baseline['mlt'].astype(float)
        baseline.dropna(inplace=True)
        plt.plot(baseline.mlt, baseline.asy, label=n, linewidth=3)
        print(baseline)
        break
    
    plt.ylim(-30,30)
    plt.axhline(y=0, color='red', linestyle='-.')
    plt.legend()
    plt.title('Asymmetry Index of the EIA Peak Magnitude')
    plt.ylabel('ASY index (%)')
    plt.xlabel('MLT (Hours)')
    plt.grid(True)
    fig.savefig(f'{work}Qingyu_Cesar_EIA_IHA/outputs/asy_ind/{month}/{n[-8:]}_{month}.jpg')
    fig.show()
    
    sc_base.append(baseline)


1it [00:00,  9.68it/s]

       mlt        asy
17  13.125  16.586575
18  13.875  18.308307
19  14.625  20.836941
20  15.375  25.547672
21  16.125  29.186294
22  16.875  33.947837
25  19.125  55.629740
26  19.875  44.253906
27  20.625  27.411532
28  21.375   2.652788
29  22.125 -16.699809
30  22.875 -28.804311
31  23.625 -40.352498
       mlt        asy
17  13.125   1.647388
18  13.875   6.326920
19  14.625  12.752935
20  15.375  17.747342
26  19.875  87.015907
27  20.625  88.778047
28  21.375  84.986434
29  22.125  71.976606
30  22.875  61.637259
31  23.625  51.196621
       mlt        asy
17  13.125   0.662995
18  13.875  -0.812731
19  14.625   0.534293
20  15.375   5.057673
21  16.125   6.023817
22  16.875   4.798135
23  17.625  13.986839
24  18.375  19.120417
25  19.125  16.879132
26  19.875  11.240315
27  20.625  14.819663
28  21.375  20.579287
29  22.125  31.505732
30  22.875  42.871644
31  23.625  54.666527


5it [00:00, 11.00it/s]

       mlt        asy
17  13.125   0.591138
18  13.875   5.105171
19  14.625  10.305222
20  15.375  15.588667
21  16.125  18.359130
22  16.875  19.224451
23  17.625  22.159412
24  18.375  31.364140
25  19.125  45.969954
26  19.875  44.636680
27  20.625  44.466976
28  21.375  45.770532
29  22.125  50.148069
30  22.875  57.968428
31  23.625  69.802444
       mlt        asy
17  13.125   0.487579
18  13.875   0.961159
19  14.625   4.221890
20  15.375   5.890474
21  16.125   6.371949
22  16.875   7.258280
23  17.625  11.403283
24  18.375  23.990721
25  19.125  38.141631
26  19.875  38.240650
27  20.625  38.425182
28  21.375  39.343232
29  22.125  44.469914
30  22.875  58.008368
31  23.625  75.092258


In [11]:
color = ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']
marker = ['o', 'x', 'D', 's']


# Solar maxima EIAs
fig = plt.figure(figsize=(12,8))
for s, name in zip(sc_base[3:], sc[3:]):
    plt.plot(s.mlt, s.asy, label=name, linewidth=3)
n = 0

for a, y in zip(asy_ind[9:], years[9:]):
    plt.plot(a.mlt, a.asy, linestyle = '--', c = color[n])
    if y < 2011:
        m = marker[0]
    elif y < 2021:
        m = marker[1]
    else:
        m = marker[2]
    plt.scatter(a.mlt, a.asy, marker = m, c = color[n], label=f'{str(y)} ASY Index', s=20)
    n = n + 1

plt.ylim(-30,30)
plt.axhline(y=0, color='red', linestyle='-.')
plt.legend()
plt.title('Asymmetry Index of the EIA Peak Magnitude')
plt.ylabel('ASY index (%)')
plt.xlim(13,22)
plt.xlabel('MLT (Hours)')
plt.grid(True)
fig.savefig(f'{work}Qingyu_Cesar_EIA_IHA/outputs/asy_ind/{month}/asy_all_MAX_{month}.jpg')
fig.show()

# Solar minima EIAs
fig = plt.figure(figsize=(12,8))
for s, name in zip(sc_base[:2], sc[:2]):
    plt.plot(s.mlt, s.asy, label=name, linewidth=3)
n = 0

for a, y in zip(asy_ind[:6], years[:6]):
    plt.plot(a.mlt, a.asy, linestyle = '--', c = color[n])
    if y < 2011:
        m = marker[0]
    elif y < 2021:
        m = marker[1]
    else:
        m = marker[2]
    plt.scatter(a.mlt, a.asy, marker = m, c = color[n], label=str(y) + ' ASY Index', s=20)
    n = n + 1

plt.ylim(-30,30)
plt.axhline(y=0, color='red', linestyle='-.')
plt.legend()
plt.title('Asymmetry Index of the EIA Peak Magnitude')
plt.ylabel('ASY index (%)')
plt.xlim(13,22)
plt.xlabel('MLT (Hours)')
plt.grid(True)
fig.savefig(f'{work}Qingyu_Cesar_EIA_IHA/outputs/asy_ind/{month}/asy_all_MIN_{month}.jpg')
fig.show()



In [12]:
color = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

# Solar Max Peak Locations
n = 0
fig = plt.figure(figsize=(12,8))
for a, y in zip(asy_ind[6:], years[6:]):
    a_n = a[a.mlat_n > 8].reset_index(drop=True)
    a_s = a[a.mlat_s < -8].reset_index(drop=True)
    if y < 2003:
        mlat = list(a_s.mlat_s)
        time = list(a_s.mlt)
    else:
        mlat = list(a_n.mlat_n) + list(a_s.mlat_s)
        time = list(a_n.mlt) + list(a_s.mlt)
        plt.plot(list(a_n.mlt), list(a_n.mlat_n), c=color[n])
    plt.scatter(time, mlat, c=color[n], label = str(y))
    plt.plot(list(a_s.mlt), list(a_s.mlat_s), c=color[n])
    n = n+1
    
if month == 'june':
    plt.ylim(-15,30)
else:
    plt.ylim(-20,20)
plt.xlim(13,22)
plt.legend()
plt.title('Position of NH SH Peaks During Solar Maximas')
plt.ylabel('MLAT')
plt.xlabel('MLT (Hours)')
plt.grid(True)
fig.savefig(f'{work}Qingyu_Cesar_EIA_IHA/outputs/asy_ind/{month}/pos_all_MAX_{month}.jpg')
fig.show()


# Solar Min Peak Locations
fig = plt.figure(figsize=(12,8))
n = 0
for a, y in zip(asy_ind[:6], years[:6]):
    a_n = a[a.mlat_n > 8].reset_index(drop=True)
    a_s = a[a.mlat_s < -8].reset_index(drop=True)
    if y < 2003:
        mlat = list(a_s.mlat_s)
        time = list(a_s.mlt)
    else:
        mlat = list(a_n.mlat_n) + list(a_s.mlat_s)
        time = list(a_n.mlt) + list(a_s.mlt)
        plt.plot(list(a_s.mlt), list(a_s.mlat_s), c=color[n])
    plt.scatter(time, mlat, c = color[n], label=str(y))
    plt.plot(list(a_n.mlt), list(a_n.mlat_n), c=color[n])
    n = n + 1

if month == 'june':
    plt.ylim(-20, 30)
else:
    plt.ylim(-15,15)
plt.xlim(13,22)
plt.legend()
plt.title('Position of NH SH Peaks During Solar Minimas')
plt.ylabel('MLAT')
plt.xlabel('MLT (Hours)')
plt.grid(True)
fig.savefig(f'{work}Qingyu_Cesar_EIA_IHA/outputs/asy_ind/{month}/pos_all_MIN_{month}.jpg')
fig.show()
